In [1]:
import pandas as pd
import os

from formatacao_sumulas import FootballDataProcessor
from algorithm import FootballMatchRating
import numpy as np
import warnings
warnings.filterwarnings("ignore")


id_players = pd.read_excel('id_players.xlsx', index_col=0)


data_folder = '../Data/'


In [2]:
id_players

,player,rating,age,games_played,player_name,time_jogador
0,328974,1500,25,0,NaN,NaN
1,407969,1500,25,0,NaN,NaN
2,595214,1500,25,0,NaN,NaN
3,612866,1500,25,0,NaN,NaN
4,138485,1500,25,0,NaN,NaN
...,...,...,...,...,...,...
14892,441035,1500,25,0,NaN,NaN
14893,184992,1500,25,0,NaN,NaN
14894,295793,1500,25,0,NaN,NaN
14895,465944,1500,25,0,NaN,NaN


In [ ]:
years = range(2014, 2024)
campeonatos = ['Serie_A', 'CdB']

for year in years:
    for campeonato in campeonatos:
        print(year, campeonato)
        df_jogos_2014 = pd.read_json(f'../Data/{campeonato}_{year}_games.json')
        df_jogos_2014_ = df_jogos_2014.T


        for index, row in df_jogos_2014_.iterrows():
            if index-1 < len(df_jogos_2014_):
                processor = FootballDataProcessor(df_jogos_2014_, index-1)
                final_df = processor.process(data_folder, campeonato, year)
                final_df.rename(columns={'player_id': 'player'}, inplace=True)
                final_df['player'] = final_df['player'].astype('string')
                id_players['player'] = id_players['player'].astype('string')

                final_df_ = pd.merge(final_df, id_players, on='player', how='left')

                team_1_name = df_jogos_2014_.iloc[index-1, 0]
                team_2_name = df_jogos_2014_.iloc[index-1, 1]

                rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

                updated_match_data = rating_calculator.update_ratings()

                teams_over_9_players = updated_match_data[updated_match_data['Minutes Played'] == 90] \
                                        .groupby('team').filter(lambda x: len(x) > 10)
                # if len(teams_over_9_players) != 0:
                #     #print(team_1_name, team_2_name, index-1)
                #     continue

                updated_match_data['player'] = updated_match_data['player'].astype('string')

                updated_match_data.rename(columns={'player_name_x': 'player_name'}, inplace=True)
                updated_match_data.rename(columns={'time_jogador_x': 'time_jogador'}, inplace=True)

                updated_values = id_players[['player']].merge(
                    updated_match_data[['player', 'rating', 'games_played', 'player_name', 'time_jogador']],
                    on='player',
                    how='left'
                )

                # Atualizar apenas as linhas corespondentes em id_players

                id_players.update(updated_values)
            else:
                print(f"Índice {index-1} fora dos limites para o ano {year}")
                continue


2014 Serie_A
2014 CdB
Índice 159 fora dos limites para o ano 2014
Índice 160 fora dos limites para o ano 2014
Índice 161 fora dos limites para o ano 2014
Índice 162 fora dos limites para o ano 2014
Índice 163 fora dos limites para o ano 2014
Índice 164 fora dos limites para o ano 2014
Índice 165 fora dos limites para o ano 2014
Índice 166 fora dos limites para o ano 2014
Índice 167 fora dos limites para o ano 2014
Índice 168 fora dos limites para o ano 2014
Índice 169 fora dos limites para o ano 2014
Índice 170 fora dos limites para o ano 2014
Índice 171 fora dos limites para o ano 2014
2015 Serie_A
2015 CdB
Índice 158 fora dos limites para o ano 2015
Índice 159 fora dos limites para o ano 2015
Índice 160 fora dos limites para o ano 2015
Índice 161 fora dos limites para o ano 2015
Índice 162 fora dos limites para o ano 2015
Índice 163 fora dos limites para o ano 2015
Índice 164 fora dos limites para o ano 2015
Índice 165 fora dos limites para o ano 2015
Índice 166 fora dos limites para

In [4]:
x = id_players.sort_values(by='rating', ascending=False)
x.to_excel('ratings.xlsx')
x

,player,rating,age,games_played,player_name,time_jogador
1093,346636,1755.026807,25,254,6Rene Rene Rodrigues Martins,Internacional / RS
6191,633571,1745.579044,25,120,15GUSTAVO GOMEZ Gustavo Raul Gomez P ...,Palmeiras / SP
12315,339258,1738.598551,25,156,13Guilherme ... Guilherme Antonio Ar ...,Atlético Mineiro / MG
125,169050,1734.646620,25,304,21Weverton Weverton Pereira da Silva,Palmeiras / SP
1550,176390,1733.823905,25,231,2Marcos Rocha Marcos Luis Rocha Aquino,Palmeiras / SP
...,...,...,...,...,...,...
7719,293325,1336.765094,25,28,2Dankler Dankler Luis de Jesu ...,Joinville / SC
11373,530656,1330.108359,25,29,4Ignacio Ignacio da Silva Oliveira,Bahia / BA
7076,510781,1325.684426,25,26,28Joaquim Joaquim Henrique Per ...,Botafogo / SP
10827,540237,1304.592011,25,20,1Joao Paulo Joao Paulo Ribeiro S ...,Chapecoense / SC


In [5]:
x.head(10)

,player,rating,age,games_played,player_name,time_jogador
1093,346636,1755.026807,25,254,6Rene Rene Rodrigues Martins,Internacional / RS
6191,633571,1745.579044,25,120,15GUSTAVO GOMEZ Gustavo Raul Gomez P ...,Palmeiras / SP
12315,339258,1738.598551,25,156,13Guilherme ... Guilherme Antonio Ar ...,Atlético Mineiro / MG
125,169050,1734.646620,25,304,21Weverton Weverton Pereira da Silva,Palmeiras / SP
1550,176390,1733.823905,25,231,2Marcos Rocha Marcos Luis Rocha Aquino,Palmeiras / SP
9030,173551,1726.614862,25,89,4Moledo Rodrigo Modesto da S ...,Internacional / RS
12730,633626,1724.192994,25,43,26Piris Robert Ayrton Piris ...,Flamengo / RJ
14349,317445,1720.855923,25,110,2Guga Claudio Rodrigues Gomes,Atlético Mineiro / MG
358,167509,1720.272773,25,223,4Rever Rever Humberto Alves ...,Atlético Mineiro / MG
8056,312950,1718.619745,25,184,12Mayke Mayke Rocha de Oliveira,Palmeiras / SP


In [6]:
unique_values_count = x['time_jogador'][:50].value_counts()
unique_values_count

time_jogador
Flamengo / RJ            18
Palmeiras / SP           14
Atlético Mineiro / MG     6
Internacional / RS        5
Fluminense / RJ           4
Vasco da Gama / RJ        1
São Paulo / SP            1
Botafogo / RJ             1
Name: count, dtype: int64

In [7]:
final_df_sorted = final_df.sort_values(by=['ano_campeonato', 'rating'], ascending=[True, False])

# Criando a coluna 'posicao' com a posição do jogador dentro de cada 'ano_campeonato'
final_df_sorted['posicao'] = final_df_sorted.groupby('ano_campeonato').cumcount() + 1
final_df_sorted

KeyError: 'ano_campeonato'

In [ ]:
#select only posicao 1
final_df_sorted[final_df_sorted['ano_campeonato'] == '2020_Serie_A']

,player,rating,player_name,time_jogador,ano_campeonato,posicao


In [ ]:
#select only posicao 1
final_df_sorted[final_df_sorted['posicao'] == 1]

,player,rating,player_name,time_jogador,ano_campeonato,posicao
5,394002,NaN,NaN,NaN,2014_CdB,1
0,394002,NaN,NaN,NaN,2014_Serie_A,1
15,394002,NaN,NaN,NaN,2015_CdB,1
10,394002,NaN,NaN,NaN,2015_Serie_A,1
